In [1]:
import openai
import os
import tiktoken
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
enc.max_token_value

100276

In [3]:
emotions = [
    "happy",
    "sad",
    "angry",
    "tired",
    "very happy",
    "very sad",
    "very angry",
    "very tired",
]

In [4]:
def create_logit_biases(emotions, bias_weight):
    """Creates a dict of tokens for ChatGPT to use mostly exclusively."""
    bias_dict = {}
    for emotion in emotions:
        token_ids = enc.encode(emotion)
        for token_id in token_ids:
            bias_dict[token_id] = bias_weight
    return bias_dict

In [5]:
emotion_list_str = [f"- {x}\n" for x in emotions]
eos_str = "."
system = f"You are an emotionally intelligent assistant. Classify the sentiment of the user's text with ONLY ONE OF THE FOLLOWING EMOTIONS:\n{''.join(emotion_list_str)}\n\nAfter classifying a text, respond with \"{eos_str}\"."


def chatgpt_sentiment(prompt):
    r = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
        stop=eos_str,
        max_tokens=10,  # safeguard from going infinite
        temperature=0.0,  # deterministic, will use largest logit
        logit_bias=create_logit_biases(
            emotions + [eos_str], 10  # may want to tweak bias weight
        ),
    )
    return r["choices"][0]["message"]["content"]


In [6]:
chatgpt_sentiment("I just bought a new iPhone.")

'happy'

In [7]:
chatgpt_sentiment("Today is a great day!")

'very happy'

In [8]:
chatgpt_sentiment("Feeling weird today.")

'tired'

In [9]:
chatgpt_sentiment("I have literally no energy.")

'very tired'

In [10]:
chatgpt_sentiment("I can't believe I got fired!")

'sad'

In [11]:
chatgpt_sentiment("My mother just died from cancer.")

'very sad'

In [12]:
chatgpt_sentiment("I am mad")

'angry'

In [13]:
chatgpt_sentiment("GOD DAMN IT")

'very angry'